In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [ ]:
df = pd.read_csv('data.csv',sep=',',encoding="ISO-8859-1")

In [ ]:
#2. Proses Filtering/Preprocessing
import numpy as np
import pandas as pd
import re
import string
import nltk

In [ ]:
# cleaning
def cleaning(str):
 #remove RT
 str = re.sub ('RT', '', str)
 #remove @username
 str = re.sub('@[^\s]+', '', str)
 str = re.sub('@', '', str)
 #remove user
 str = re.sub ('USER', '', str)
 #remove hashtag
 str = re.sub('#[^\s]+', '', str)
 str = re.sub('#', '', str)
 #remove URLs
 str = re.sub(r'https?://\S+|www\.\S+', '', str)
 #remove URL word
 str = re.sub ('URL', '', str)
 #remove digit from string
 str = re.sub("\S*\d\S*", "", str).strip()
 #remove digit or numbers
 str = re.sub(r"\b\d+\b", " ", str)
 #Remove additional white spaces
 str = re.sub('\s+', ' ', str)
 #to lowercase
 str = str.lower()
    
 return str 

In [ ]:
df['text'] = df['Tweet'].apply(cleaning)

In [ ]:
df['text'].loc[288]

In [ ]:
df['text'].replace('', np.nan, inplace=True)

In [ ]:
df['text'].isna().sum()

In [ ]:
df.dropna(subset=['text'], inplace=True)

In [ ]:
df['text'].isna().sum()

In [ ]:
#----------------Tokenizing----------------#
import string
import re
from nltk.tokenize import word_tokenize
x = df.iloc[0]
print(nltk.word_tokenize(x['text']))
def identify_tokens(row):
 message = row['text']
 tokens = nltk.word_tokenize(message)
 token_words = [w for w in tokens if w.isalpha()]
 return token_words

In [ ]:
df['text'] = df.apply(identify_tokens, axis = 1)
df.text

In [ ]:
#--------------Filtering-------------#
#stopword
from nltk.corpus import stopwords
stops = set(stopwords.words('indonesian'))
def remove_stops(row):
 message = row['text']
 stopwords = [w for w in message if not w in stops]
 return stopwords

In [ ]:
df['text'] = df.apply(remove_stops, axis = 1)
df['text'] = df['text'].str.join(" ")
df.text

In [ ]:
##singkatan ##normalisasi

In [ ]:
##
abbreviations = dict()
with open('new_kamusalay.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        abbreviation = row[0].strip()
        expansion = row[1].strip()
        abbreviations[abbreviation] = expansion

In [ ]:
##
abbreviations

In [ ]:
##
def replace_abbreviations(text, abbreviations):
    words = text.split()
    for i, word in enumerate(words):
        if word in abbreviations:
            words[i] = abbreviations[word]
    return ' '.join(words)

In [ ]:
##
df['abbreviations'] = df['text'].apply(lambda text: replace_abbreviations(text, abbreviations))

In [ ]:
df['abbreviations']

In [ ]:
##stemming sastrawi

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
# Fungsi untuk melakukan stemming pada teks
def stem_text(text):
    return stemmer.stem(text)

In [ ]:
# Melakukan stemming pada kolom 'text' dalam DataFrame
df['stemmed_text'] = df['abbreviations'].apply(stem_text)

In [ ]:
df['stemmed_text']

In [ ]:
##sensor kata
abusive_words = []
import csv
with open('abusive.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        abusive_words.append(row[0].strip())

In [ ]:
abusive_words

In [ ]:
def censor_text(text, abusive_words):
    censored_text = text
    for word in abusive_words:
        censored_text = re.sub(r'\b' + re.escape(word) + r'\b', '*' * len(word), censored_text, flags=re.IGNORECASE)
    return censored_text

In [ ]:
df['Censored_Text'] = df['stemmed_text'].apply(lambda x: censor_text(x, abusive_words))

In [ ]:
df['Censored_Text']

In [ ]:
#
def hatesp(hs):
    kategori = ""
    if int(hs) == 1:
        hatesp = 'hate speech'
    else:
        hatesp = 'non-hate speech'
    return hatesp

In [ ]:
df['kategori_hs'] = df['HS'].apply(hatesp)
df[['stemmed_text', 'kategori_hs']].sample(10)

ANALISIS

In [ ]:
#Comparasion Hate Speech on Tweets Data
fig, ax = plt.subplots(figsize = (3,8))
sizes = [count for count in df['kategori_hs'].value_counts()]
labels = list(df['kategori_hs'].value_counts().index)
ax.pie(x = sizes, labels = labels, autopct = '%1.1f%%', textprops={'fontsize': 14})
ax.set_title('Category Hate Speech', fontsize = 16, pad = 20)
plt.show()

In [ ]:
df['kategori_hs'].value_counts()

In [ ]:
plt.figure(figsize=(4,4))
sns.countplot(x='kategori_hs',data=df)
plt.title('Category Hate Speech')

In [ ]:
#histogram of tweet lengths
df['tweet_length'] = df['stemmed_text'].apply(len)
plt.figure(figsize=(3,3))
sns.histplot(df['tweet_length'], bins=50)
plt.xlabel('All Tweets')
plt.ylabel('Frequency')
plt.title('Distribution of Tweet Character Length')
plt.show()

In [ ]:
#Distribution of Tweet Lengths #All tweets
df['word_length'] = df['stemmed_text'].apply(lambda sent : len(sent.split()))
plt.figure(figsize=(3,3))
sns.histplot(df['word_length'], bins=50)
plt.xlabel('All Tweets')
plt.ylabel('Frequency')
plt.title('Distribution of Tweet Word Length')
plt.show()

In [ ]:
hs_length = df['stemmed_text'][df['kategori_hs']=='hate speech']
hsnon_length = df['stemmed_text'][df['kategori_hs']=='non-hate speech']

In [ ]:
#Distribution of Tweet Character Lengths #Hate Speech
df['tweet_length_hs'] = hs_length.apply(len)
plt.figure(figsize=(3,3))
sns.histplot(df['tweet_length_hs'], bins=50)
plt.xlabel('Hate Speech Tweets')
plt.ylabel('Frequency')
plt.title('Distribution of Tweet Character Length')
plt.show()

In [ ]:
#Distribution of Tweet Lengths #Hate Speech
df['word_length_hs'] = hs_length.apply(lambda sent : len(sent.split()))
plt.figure(figsize=(3,3))
sns.histplot(df['word_length_hs'], bins=50)
plt.xlabel('Hate Speech Tweets')
plt.ylabel('Frequency')
plt.title('Distribution of Tweet Word Length')
plt.show()

In [ ]:
#Distribution of Tweet Lengths #Non Hate Speech
df['tweet_length_nhs'] = hsnon_length.apply(len)
plt.figure(figsize=(3,3))
sns.histplot(df['tweet_length_nhs'], bins=50)
plt.xlabel('Non Hate Speech Tweets')
plt.ylabel('Frequency')
plt.title('Distribution of Tweet Character Length')
plt.show()

In [ ]:
#Distribution of Tweet Lengths #Non Hate Speech
df['word_length_nhs'] = hsnon_length.apply(lambda sent : len(sent.split()))
plt.figure(figsize=(3,3))
sns.histplot(df['word_length_nhs'], bins=50)
plt.xlabel('Non Hate Speech Tweets')
plt.ylabel('Frequency')
plt.title('Distribution of Tweet Word Length')
plt.show()

In [ ]:
#korelasi panjang kata dan karakter pada suatu tweet

In [ ]:
plt.figure(figsize=(3,3))
sns.histplot(x = df['word_length'], y = df['tweet_length'], bins=50)
plt.xlabel('Total Word (All Tweets)')
plt.ylabel('Total Character')
plt.title('Correlation Total Word & Total Character')
plt.show()

In [ ]:
plt.figure(figsize=(3,3))
sns.histplot(x = df['word_length_hs'], y = df['tweet_length_hs'], bins=50)
plt.xlabel('Total Word (Hate Speech)')
plt.ylabel('Total Character')
plt.title('Correlation Total Word & Total Character')
plt.show()

In [ ]:
plt.figure(figsize=(3,3))
sns.histplot(x = df['word_length_nhs'], y = df['tweet_length_nhs'], bins=50)
plt.xlabel('Total Word (Non Hate Speech)')
plt.ylabel('Total Character')
plt.title('Correlation Total Word & Total Character')
plt.show()

In [ ]:
#wordcloud masih mengandung kata yang tidak penting
from wordcloud import WordCloud
text = ' '.join(df['stemmed_text'])
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
#hapus kata abusive
abusive_words = []
import csv
with open('abusive.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        abusive_words.append(row[0].strip())

In [ ]:
def remove_abusive_words(text, abusive_words):
    # Gabungkan daftar kata abusive menjadi pola regex
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in abusive_words) + r')\b'
    # Hapus kata-kata abusive dari teks
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return cleaned_text

In [ ]:
df['perbaikan_kata'] = df['stemmed_text'].apply(lambda x: remove_abusive_words(x, abusive_words))

In [ ]:
#menghapus stopword yang masih ada
daftar_stopword = pd.read_csv('stopwordbahasa.csv', names = ['stopword'])
daftar_stopword

In [ ]:
text_baru = ' '.join(df['perbaikan_kata'])

In [ ]:
text_baru

In [ ]:
split_text = text_baru.split()
filtered_array = [item for item in split_text if item not in daftar_stopword['stopword'].values]
filtered_text = ' '.join(filtered_array)

In [ ]:
filtered_text 

In [ ]:
#wordcloud data keseluruhan
from wordcloud import WordCloud
wordcloud = WordCloud().generate(filtered_text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
from collections import Counter
# Menghitung frekuensi kemunculan kata dengan menggunakan Counter
word_counts = Counter(filtered_text.split())
# Menampilkan 10 kata yang paling banyak muncul beserta frekuensinya
top_words = word_counts.most_common(10)
top_words

In [ ]:
# Membuat dataframe dari top_words
df_topwords = pd.DataFrame(top_words, columns=['word', 'count'])

# Membuat bar plot dari dataframe dengan matplotlib
plt.figure(figsize=(4,4))
plt.barh(df_topwords['word'], df_topwords['count'], color='blue')
plt.title('Top 10 Words')
plt.xlabel('Word')
plt.ylabel('Count')
plt.show()

In [ ]:
hs_sp = df['perbaikan_kata'][df['kategori_hs']=='hate speech']
hs_non = df['perbaikan_kata'][df['kategori_hs']=='non-hate speech']

In [ ]:
text_hs = ' '.join(hs_sp)
split_hs = text_hs.split()
filtered_array_hs = [item for item in split_hs if item not in daftar_stopword['stopword'].values]
filtered_text_hs = ' '.join(filtered_array_hs)

In [ ]:
wordcloud_hs = WordCloud().generate(filtered_text_hs)
plt.imshow(wordcloud_hs)
plt.axis("off")
plt.show()

In [ ]:
from collections import Counter
# Menghitung frekuensi kemunculan kata dengan menggunakan Counter
word_counts_hs = Counter(filtered_text_hs.split())
# Menampilkan 10 kata yang paling banyak muncul beserta frekuensinya
top_words_hs = word_counts_hs.most_common(10)
top_words_hs

In [ ]:
# Membuat dataframe dari top_words
df_topwords_hs = pd.DataFrame(top_words_hs, columns=['word', 'count'])

# Membuat bar plot dari dataframe dengan matplotlib
plt.figure(figsize=(4,4))
plt.barh(df_topwords_hs['word'], df_topwords_hs['count'], color='blue')
plt.title('Top 10 Words of Hate Speech')
plt.xlabel('Word')
plt.ylabel('Count')
plt.show()

In [ ]:
text_nhs = ' '.join(hs_non)
split_nhs = text_nhs.split()
filtered_array_nhs = [item for item in split_nhs if item not in daftar_stopword['stopword'].values]
filtered_text_nhs = ' '.join(filtered_array_nhs)

In [ ]:
wordcloud_nhs = WordCloud().generate(filtered_text_nhs)
plt.imshow(wordcloud_nhs)
plt.axis("off")
plt.show()

In [ ]:
from collections import Counter
# Menghitung frekuensi kemunculan kata dengan menggunakan Counter
word_counts_nhs = Counter(filtered_text_nhs.split())
# Menampilkan 10 kata yang paling banyak muncul beserta frekuensinya
top_words_nhs = word_counts_nhs.most_common(10)
top_words_nhs

In [ ]:
# Membuat dataframe dari top_words
df_topwords_nhs = pd.DataFrame(top_words_nhs, columns=['word', 'count'])

# Membuat bar plot dari dataframe dengan matplotlib
plt.figure(figsize=(4,4))
plt.barh(df_topwords_nhs['word'], df_topwords_nhs['count'], color='blue')
plt.title('Top 10 Words of Non-Hate Speech')
plt.xlabel('Word')
plt.ylabel('Count')
plt.show()

In [ ]:
df['HS_Individual'].value_counts()

In [ ]:
macam_hs = df[['HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other']]

In [ ]:
macam_hs

In [ ]:
plt.figure(figsize=(4,4))
sns.countplot(data=macam_hs)
plt.title('Category Hate Speech')